In [ ]:
import csv
import random
import re
import time
import psycopg2
import itertools
# import csv_utils
import requests
import bs4
from unidecode import unidecode

CSV_FILE_ENCODING = 'utf-8'
# global municodemap
# global current_muni
# global MUNI_PARCELID_LIST_FILE
# global muni_idbase_map
# global ID_BASE
# global TEST_PARID_FILE

def main():
    # for accessing a list of all parcelIDs by with one muni's list per file
    # global INPUT_FILE
    # INPUT_FILE = 'workspace/pitcairnparids.csv'

    # parcels whose base property data is not written are written here before moving on
    global LOG_FILE
    LOG_FILE = 'output/swissvaleerror.csv'

    # this log file is used for storing parcels whose person and propertyperson inserts fail
    global LOG_FILE_AUX
    LOG_FILE_AUX = 'output/swissvaleerror_aux.csv'

    # ID number of the system user connected to these original inserts
    # user 99 is sylvia, our COG robot
    global UPDATING_USER_ID
    UPDATING_USER_ID = 99

    global PARID_FILE
    # PARID_FILE = 'parcelidlists/wilmerdingparids.csv'
    # PARID_FILE = 'parcelidlists/parcelidstest.csv'
    #PARID_FILE = 'parcelidlists/pitcairnparids_correct.csv'
    #PARID_FILE = 'parcelidlists/eastmckeesportparids.csv'
    #PARID_FILE = 'parcelidlists/wilkinsparcelids.csv'
    #PARID_FILE = 'parcelidlists/chalfantparcelids.csv'
    PARID_FILE = 'parcelidlists/swissvaleparcelids.csv'
    
    # used as the access key for muni codes and ID bases in the dictionaries below
    global current_muni
    current_muni = 'swissvale'

    # use as floor value for all new propertyIDs
    global PROP_ID_BASE
    PROP_ID_BASE = 100000

    # floor value for new personIDs
    global PERSON_ID_BASE
    PERSON_ID_BASE = 10000

    # used for sliding starting ID up to accommodate a parcelid list adjustment due to errors on pesky parcels
    # must be manually set if there's an error: start one above the most recently issued ID
    # this gets added to the base ID in the ID range generation methods
    global BUMP_UP
    BUMP_UP = 0

    global municodemap
    municodemap = {'chalfant':814,'churchhill':816, 'eastmckeesport':821, 'pitcairn':847, 'wilmerding':867, 'wilkins':953, 'cogland':999, 'swissvale':111}

    # add these base amounts to the universal base to get starting IDs
    global muni_idbase_map
    muni_idbase_map = {'chalfant':10000,'churchhill':20000, 'eastmckeesport':30000, 'pitcairn':40000, 'wilmerding':0, 'wilkins':50000, 'cogland':60000, 'swissvale':70000}

    # add these base amounts to the universal base to get starting IDs
    global person_idbase_map
    person_idbase_map = {'chalfant':10000,'churchhill':20000, 'eastmckeesport':30000, 'pitcairn':40000, 'wilmerding':0, 'wilkins':50000, 'cogland':60000, 'swissvale':70000}

    # jump into the actual work here
    insert_property_basetableinfo()
    
    # Add owner data to properties
    # properties_with_owner_info = add_prop_info(properties)
    # Save new CSV file with property plus owner data
    # OUTPUT_FILE = 'output/testPropOut.txt'
    # save_properties_as_csv(properties_with_owner_info, OUTPUT_FILE)

def get_nextpropertyid(munioffset):
    # consider a range multiplier by municipality to generate starting 
    # at, say 110000 and the next at 120000 which allows for non-overlapping
    #ids for munis with a property count of up to 10000
    for i in list(range(PROP_ID_BASE + munioffset + BUMP_UP, PROP_ID_BASE + munioffset + 9999)):
        yield i;

def get_nextpersonid(munioffset):
    for i in list(range(PERSON_ID_BASE + munioffset + BUMP_UP, PERSON_ID_BASE + munioffset + 9999)):
        yield i;



def get_nextparcelid(input_file):
    # Read the CSV file and iterate through every property
    with open(input_file, 'r', encoding=CSV_FILE_ENCODING) as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            parid = str(row['parcelid']).strip()
            yield parid


def logerror(parcelid):
    with open(LOG_FILE, 'a', encoding=CSV_FILE_ENCODING) as outfile:
        writer = csv.writer(outfile)
        writer.writerow([parcelid])

def logerror_aux(parcelid):
    with open(LOG_FILE_AUX, 'a', encoding=CSV_FILE_ENCODING) as outfile:
        writer = csv.writer(outfile)
        writer.writerow([parcelid])

# for debugging and transferability purposes
def save_properties_as_csv(properties_with_owner_info, output_file):
    fieldnames = None
    with open(output_file, 'w', encoding=CSV_FILE_ENCODING) as outfile:
        writer = csv.writer(outfile)
        for prop in properties_with_owner_info:
            if fieldnames is None:
                fieldnames = sorted(prop.keys())
                writer.writerow(fieldnames)
            writer.writerow([prop[k] for k in fieldnames])

            
def connect_person_to_property(propertyid, personid):
    db_conn = get_db_conn()
    cursor = db_conn.cursor()

    insert_sql = """
        INSERT INTO public.propertyperson(
            property_propertyid, person_personid)
    VALUES (%(prop)s, %(pers)s);
    """
    insertmap = {}
    
    # load up vars for use in SQL from each of the parse methods
    insertmap['prop'] = propertyid
    insertmap['pers'] = personid

    cursor.execute(insert_sql, insertmap)
    db_conn.commit()
    print('----- connected person owner to property -----')

def create_and_insert_unitzero(propertyid):
    db_conn = get_db_conn()
    cursor = db_conn.cursor()
    
    insert_sql = """
        INSERT INTO public.propertyunit(
            unitid, unitnumber, property_propertyid, otherknownaddress, notes, 
            rental)
        VALUES (DEFAULT, '0', %(property_propertyid)s, NULL, 
            'robot-generated unit representing the primary habitable dwelling on a property', 
            FALSE);
    """
    insertmap = {}
    
    # load up vars for use in SQL from each of the parse methods
    insertmap['property_propertyid'] = propertyid

    cursor.execute(insert_sql, insertmap)
    db_conn.commit()
    print('----- built unit zero -----')


# deprecated from Daniel--kept in as a reference
def add_prop_info(properties):
    PARCEL_ID_FIELD = 'parcelid'
    OWNER_NAME_FIELD = 'OwnerName'
    OWNER_ADDRESS_FIELD = 'OwnerAddress'
    for prop in properties:
        assert prop[PARCEL_ID_FIELD]  # Every property must have a parcel id
        parcel_id = prop[PARCEL_ID_FIELD]
        print('Processing parcel', parcel_id)

        # Get the html from the county's website
        property_html = get_county_page_for(parcel_id)

        # Wait between request, just to be nice with the county's site
        sleep_time = random.random() * 3  # Sleep at most 3 seconds
        time.sleep(sleep_time)
        print('sleeping for {:.2f} seconds'.format(sleep_time))

        # Add owner name and address to the property
        prop[OWNER_NAME_FIELD] = extract_owner_name(parcel_id, property_html)
        prop[OWNER_ADDRESS_FIELD] = extract_owner_address(
                parcel_id, property_html)
        print(prop[OWNER_NAME_FIELD], prop[OWNER_ADDRESS_FIELD])

        yield prop


county_info_cache = {}
def get_county_page_for(parcel_id):
    if parcel_id in county_info_cache:
        return county_info_cache[parcel_id]
    COUNTY_REAL_ESTATE_URL = ('http://www2.county.allegheny.pa.us/'
                              'RealEstate/GeneralInfo.aspx?')
    search_parameters = {
        'ParcelID': parcel_id,
        'SearchType': 3,
        'SearchParcel': parcel_id}
    waittime = random.uniform(0.0,1.0)
    print("waiting:" + str(waittime))
    time.sleep(waittime)
    try:
        response = requests.get(
                COUNTY_REAL_ESTATE_URL,
                params=search_parameters,
                timeout=5)
        print('Scraping data from county: %s' + parcel_id)
    except requests.exceptions.Timeout:
        # Wait 10 secs and try one more time
        time.sleep(10)
        response = requests.get(
                COUNTY_REAL_ESTATE_URL,
                params=search_parameters,
                timeout=5)
    county_info_cache[parcel_id] = response.text
    return response.text

#---------------------------------------------
#           HTML SCRAPING METHODS
#---------------------------------------------

def extract_lotandblock_fromparid(parid):
    trimmedparid = parid[0:11]
    exp = re.compile(r'([1-9]+)(\w)[0]*([1-9]+)')
    gl = re.search(exp, trimmedparid)
    if(gl):
        lob = gl.group(1) + '-' + gl.group(2) + '-' + gl.group(3)
        return lob
    else:
        print('ERROR: LOB parsing')
        return ''

def extract_owner_name(property_html):
    OWNER_NAME_SPAN_ID = 'BasicInfo1_lblOwner'
    persondict = {}

    soup = bs4.BeautifulSoup(property_html, 'lxml')
    owner_full_name = soup.find('span', id=OWNER_NAME_SPAN_ID).text
    print('owner_raw_name:' + str(owner_full_name))
    # Remove extra spaces from owner's name
    full_owners = re.sub(r'\s+', ' ', owner_full_name.strip())
    
    #exp = re.compile(r'(\w+|[&])\s+(\w+|[&])\s*(\w*|[&]).*')
    #namegroups = re.search(exp, owner_full_name)
    #print(len(namegroups.groups()))
    #persondict['lname'] = str(namegroups.group(1)).title()
    
    #if len(namegroups.groups()) == 2:
        #persondict['fname'] = str(namegroups.group(2)).title()
    #elif len(namegroups.groups()) == 3:
        #persondict['fname'] = str(namegroups.group(2)).title() + ' ' + str(namegroups.group(3)).title()

    #print(str(persondict['fname']) + ' ' + str(persondict['lname']))
    
    return full_owners

def extract_propertyaddress(parcel_id, property_html):
    propaddrmap = {}
    PROP_ADDR_SPAN_ID = 'BasicInfo1_lblAddress'
    # print(property_html)
    soup = bs4.BeautifulSoup(property_html, 'lxml')
    
    # this yeilds something like
    # 471&nbsp;WALNUT  ST<br>PITTSBURGH,&nbsp;PA&nbsp;15238
    prop_addr_raw = soup.find('span', id=PROP_ADDR_SPAN_ID).text
    scrapedhtml = soup.select("#BasicInfo1_lblAddress")

    print("Scraped:" + str(scrapedhtml))
    soup = bs4.BeautifulSoup(str(scrapedhtml), 'lxml')
    adrlistraw = soup.span.contents 
    # make sure we have all the parts of the address
    if len(adrlistraw) < 3:
        print("Raising Exception in extract_propertyaddress")
        raise Exception
    
    street = re.sub('  ', ' ', adrlistraw[0])
    propaddrmap['street'] = str(unidecode(street)).strip()
    print(propaddrmap['street'])
    # on the city, state, zip line, grab until the comma before the state
    exp = re.compile('[^,]*')
    propaddrmap['city'] = exp.search(adrlistraw[2]).group()
    print("city:" + propaddrmap['city'])
    # hard-code pa
    propaddrmap['state']= 'PA'
    # zip is just the last 5 chars    
    propaddrmap['zipc'] = adrlistraw[2][-5:]
    print("Zip:" + propaddrmap['zipc'])

    return propaddrmap

def extract_owneraddress(property_html):
    print('--------- extracting owner address ------------')
    owneraddrmap = {}
    # print(property_html)
    soup = bs4.BeautifulSoup(property_html, 'lxml')
    # this yeilds something like
    # 471&nbsp;WALNUT  ST<br>PITTSBURGH,&nbsp;PA&nbsp;15238
    scrapedhtml = soup.select("#lblChangeMail")
    owneraddrmap['notes_adrdump'] = str(scrapedhtml)

    print("Scraped:" + str(scrapedhtml))
    soup = bs4.BeautifulSoup(str(scrapedhtml), 'lxml')
    # this spits out a three-item list. We throw away the <br> which is 
    # the middle item. The first is the street, the second gets chopped up
    adrlistraw = soup.span.contents 
    # make sure we have all the parts of the address
    if len(adrlistraw) < 3:
        print("Raising Exception")
        raise Exception
    
    addrlistindex = 0
    if len(adrlistraw) > 3:
        print("Extra line address")
        addrlistindex += 2
        
    ownerstreet = re.sub('  ', ' ', adrlistraw[addrlistindex])
    owneraddrmap['street'] = str(unidecode(ownerstreet)).strip()
    print(owneraddrmap['street'])
    # on the city, state, zip line, grab until the comma before the state
    exp = re.compile('[^,]*')
    addrlistindex += 2
    owneraddrmap['city'] = exp.search(adrlistraw[addrlistindex]).group()
    print("city:" + owneraddrmap['city'])
    # grap with string indexes (fragile if there is more than one space before zip)
    # TODO: use regexp
    exp=re.compile(r',\s*(\w\w)')
    m = re.search(exp,adrlistraw[addrlistindex])
    owneraddrmap['state'] = str(m.group(1))
    
    # abandoned string slicing approach (too brittle; use regexp)
    # owneraddrmap['state']= adrlistraw[2][-13:-11]
    print("state:" + owneraddrmap['state'])
    
    # owner zips could come in as: 15218 OR 15218- OR 15218-2342
    # just lose the routing numbers and take the first digits until the -
    exp=re.compile(r'\d+')
    m = re.search(exp, adrlistraw[addrlistindex])
    owneraddrmap['zipc'] = str(m.group())

    # another abandoned string slicing approach: 
    # also too brittle given range of scraped inputs
    # owneraddrmap['zip'] = adrlistraw[2][-10:]
    print("zip:" + owneraddrmap['zipc'])

    return owneraddrmap


def extact_owner_name_and_mailing(parcel_id, property_html):

    OWNER_ADDRESS_SPAN_ID = 'lblChangeMail'
    addrparts = {}

    soup = bs4.BeautifulSoup(property_html, 'lxml')
    owner_address_raw = soup.find('span', id=OWNER_ADDRESS_SPAN_ID).text
    # Remove extra spaces
    addrparts['addr'] = rs.sub('')

    owner_address = re.sub(r'\s+', ' ', owner_address.strip())
    # Remove leading spaces before commas
    owner_address = re.sub(r'\s+,', ',', owner_address)

    return owner_address


def extract_propertyusetype(property_html):
    USE_TYPE_SPAN_ID = 'lblUse'

    soup = bs4.BeautifulSoup(property_html, 'lxml')
    usetype = soup.find('span', id=USE_TYPE_SPAN_ID).text
    # Remove extra spaces
    usetype = re.sub(r'\s+', ' ', usetype.strip())
    # Remove leading spaces before commas
    usetype = str(re.sub(r'\s+,', ',', usetype)).title()
    return usetype

def extract_class(property_html):
    CLASS_SPAN_ID = 'lblState'

    soup = bs4.BeautifulSoup(property_html, 'lxml')
    propclass = soup.find('span', id=CLASS_SPAN_ID).text
    # Remove extra spaces
    propclass = re.sub(r'\s+', ' ', propclass.strip())
    # Remove leading spaces before commas
    propclass = str(re.sub(r'\s+,', ',', propclass)).title()
    return propclass

def extract_ownercode(property_html):
    OWNER_CODE_SPAN_ID = 'lblOwnerCode'

    soup = bs4.BeautifulSoup(property_html, 'lxml')
    ownercode = soup.find('span', id=OWNER_CODE_SPAN_ID).text
    # Remove extra spaces
    ownercode = re.sub(r'\s+', ' ', ownercode.strip())
    # Remove leading spaces before commas
    ownercode = str(re.sub(r'\s+,', ',', ownercode)).title()
    return ownercode

def check_current_owner(parcelid):
    namemap = {}
    get_propid = "SELECT propertyid FROM public.property WHERE parid=%(parcelid)s;"
    inserts = {}
    inserts['parcelid'] = parcelid
    db_conn = get_db_conn()
    cursor = db_conn.cursor()
    cursor.execute(get_propid, inserts)
    propidlist = cursor.fetchall()
    propid = propidlist[0][0]
    get_personid = "SELECT person_personid FROM public.propertyperson WHERE property_propertyid=%(property_propertyid)s;"
    inserts['property_propertyid'] = propid
    cursor.execute(get_personid, inserts)
    personidlist = cursor.fetchall()
    personid = personidlist[0][0]
    get_names = "SELECT fname, lname FROM public.person WHERE personid=%(personid)s"
    inserts['personid'] = personid
    cursor.execute(get_names, inserts)
    names = cursor.fetchall()
    print(names)
    namemap['fname'] = names[0][0]
    namemap['lname'] = names[0][1]
    return namemap

def extract_and_insert_person(rawhtml, propertyid, personid, propinserts):
    # fixed values specific to keys in lookup tables
    

    db_conn = get_db_conn()
    cursor = db_conn.cursor()

    notemsg = """In case of confusion, check autmated record entry with raw text from the county database:"""

    insert_sql = """
        INSERT INTO public.person(
            personid, persontype, muni_municode, fname, lname, jobtitle, 
            phonecell, phonehome, phonework, email, address_street, address_city, 
            address_state, address_zip, notes, lastupdated, expirydate, isactive, 
            isunder18, humanverifiedby,compositelname, mailing_address_street, mailing_address_city,
            mailing_address_state, mailing_address_zip, useseparatemailingaddr)
    VALUES (%(personid)s, cast ( 'ownercntylookup' as persontype), 
            %(muni_municode)s, NULL, %(lname)s, 'Property Owner', 
            NULL, NULL, NULL, NULL, %(address_street)s, %(address_city)s, 
            %(address_state)s, %(address_zip)s, %(notes)s, now(), NULL, TRUE, 
            FALSE, NULL, true, %(mailing_street)s, %(mailing_city)s, %(mailing_state)s, %(mailing_zip)s,
            %(mailingsameasres)s);
    """
    
    
    """INSERT INTO public.person(
	personid, persontype, muni_municode, fname, lname, jobtitle, phonecell, phonehome, phonework, 
    email, address_street, address_city, address_state, address_zip, notes, lastupdated, expirydate, 
    isactive, isunder18, humanverifiedby, compositelname, sourceid, creator, businessentity, 
    mailing_address_street, mailing_address_city, mailing_address_zip, mailing_address_state, 
    useseparatemailingaddr, expirynotes, creationtimestamp, canexpire, userlink)
	VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"""
    
    insertmap = {}
    
    # load up vars for use in SQL from each of the parse methods
    insertmap['personid'] = personid
    print("personid:"+str(insertmap['personid']))
    insertmap['muni_municode'] = str(municodemap[current_muni])
    
    insertmap['mailing_street'] = None
    insertmap['mailing_city'] = None
    insertmap['mailing_state'] = None
    insertmap['mailing_zip'] = None
    insertmap['mailingsameasres'] = 'true'
    
    try:
        ownername = extract_owner_name(rawhtml)
        insertmap['lname'] = ownername
        print('extracted owner:' + insertmap['lname'] )

    except Exception:
        print("Malformed owner name at e and i person")
        print("ERROR malformed address at:" + str(propertyid))
    
    try:
        owneraddrmap = extract_owneraddress(rawhtml)
        print("ARE THEY THE SAME??? " + str(owneraddrmap['street'] == propinserts['street']))
        print(propinserts)
        if owneraddrmap['street'] == propinserts['street']:
            print("SAME ADDRESS")
            insertmap['address_street'] = owneraddrmap['street']
            insertmap['address_city'] = owneraddrmap['city']
            insertmap['address_state'] = owneraddrmap['state']
            insertmap['address_zip'] = owneraddrmap['zipc']
        else:
            print("DIFFERENT MAILING")
            insertmap['address_street'] = None
            insertmap['address_city'] = None
            insertmap['address_state'] = None
            insertmap['address_zip'] = None
            insertmap['mailing_street'] = owneraddrmap['street']
            insertmap['mailing_city'] = owneraddrmap['city']
            insertmap['mailing_state'] = owneraddrmap['state']
            insertmap['mailing_zip'] = owneraddrmap['zipc']
            insertmap['mailingsameasres'] = 'false'
        insertmap['notes'] = notemsg + ownername + " Raw address: " + owneraddrmap['notes_adrdump']

    except Exception:
        print("NO MAILING")
        insertmap['address_street'] = propinserts['street']
        insertmap['address_city'] = propinserts['city']
        insertmap['address_state'] = propinserts['state']
        insertmap['address_zip'] = propinserts['zipc']
        insertmap['notes'] = "Owner does not have mailing address"

        
    print(insertmap['mailing_zip'])
    if insertmap['address_street']:
        print('extracted owner address:' \
            + ' ' + insertmap['address_street'] \
            + ' ' + insertmap['address_city'] \
            + ', ' + insertmap['address_state'] \
            + ' ' + insertmap['address_zip'])
    if insertmap['mailing_street']:
        print('extracted mailing address:' \
            + ' ' + insertmap['mailing_street'] \
            + ' ' + insertmap['mailing_city'] \
            + ', ' + insertmap['mailing_state'] \
            + ' ' + insertmap['mailing_zip'])

    
    print('Inserting person data for id: %s' % (insertmap['personid']))
    print('person notes:' + insertmap['notes'])
    
    cursor.execute(insert_sql, insertmap)
    db_conn.commit()
    print('----- committed person owner -----')
    # commit core propertytable insert
def insert_property_basetableinfo():
    db_conn = get_db_conn()
    cursor = db_conn.cursor()
    parcelgenerator = get_nextparcelid(PARID_FILE)
    propertyidgenerator = get_nextpropertyid(muni_idbase_map[current_muni])
    personidgenerator = get_nextpersonid(person_idbase_map[current_muni])

    # user 99 is the cog robot, Sylvia
    insert_sql = """
        INSERT INTO public.property(
            propertyid, municipality_municode, parid, address, 
            notes, addr_city, addr_state, addr_zip,
            lotandblock, propclass, propertyusetype, ownercode,
            lastupdated, lastupdatedby)
        VALUES (%(propid)s, %(muni)s, %(parcelid)s, %(addr)s, 
                %(notes)s, %(city)s, %(state)s, %(zipcode)s,
                %(lotandblock)s, %(propclass)s, %(propertyusetype)s, %(ownercode)s,
                now(), %(updatinguser)s);

    """
    insertmap = {}
    propertycount = 0
    personcount = 0
    # the main delegator loop
    for parid in parcelgenerator:
        # go get raw HTML
        rawhtml = get_county_page_for(parid)
        
        # load up vars for use in SQL from each of the parse methods
        insertmap['propid'] = next(propertyidgenerator)
        propid = insertmap['propid']
        print("newid:"+ str(propid))
        # parid comes from the iterated item variable parid
        insertmap['parcelid'] = parid
        insertmap['muni'] = municodemap[current_muni]
        try:
            addrmap = extract_propertyaddress(parid, rawhtml)
        except Exception:
            print("MA base table")
            print("ERR malformed address at:" + parid)
            logerror(parid)
            continue
        insertmap['addr'] = addrmap['street']
        insertmap['notes'] = 'core data scraped from county site'
        insertmap['city'] = addrmap['city']
        insertmap['state'] = addrmap['state']
        insertmap['zipcode'] = addrmap['zipc']   
        insertmap['lotandblock'] = extract_lotandblock_fromparid(parid)
        # print('lob:' + insertmap['lotandblock'])

        insertmap['propclass'] = str(extract_class(rawhtml))
        # print('class:' + insertmap['propclass'])

        insertmap['propertyusetype'] = extract_propertyusetype(rawhtml)
        # print('use:' + insertmap['propertyusetype'])

        insertmap['ownercode'] = extract_ownercode(rawhtml)
        # print('owner code:' + insertmap['ownercode'])

        insertmap['updatinguser'] = str(99)
        # print('updater id:' + insertmap['updatinguser'])

        print('Inserting parcelid data: %s' % (parid))
        
        try:
            # execute insert on property table
            cursor.execute(insert_sql, insertmap)
            # commit core propertytable insert
            db_conn.commit()
            propertycount = propertycount + 1
            print('----- committed property core table -----')
        except:
            print('ERROR: unable to insert base property data...skipping\n')
            print('********* MOVING ON ********************')
            logerror_aux(parid)
            continue
        try:
        # this try catches soup related errors
        # and sql errors bubbling up from the extraction methods that also commit
            personid = next(personidgenerator)
            print(1)
            extract_and_insert_person(rawhtml, propid, personid, addrmap)
            print(2)
            connect_person_to_property(propid, personid)
            print(3)
            personcount = personcount + 1
        except:
            print('ERROR: unable to extract, commit, or connect person owner\n')
            logerror_aux(parid)
            continue
        try:
            # create standard unit number 0 for each property in system
            create_and_insert_unitzero(propid)
        except:
            print('ERROR: unable to create unit zero for property no.\n'+ str(propid))
            logerror(parid)
            continue
        print('--------- running totals --------')
        print('Props inserted: ' + str(propertycount))
        print('Persons inserted: ' + str(personcount))
        print('********** DONE! *************\n')

        # run insert with sql statement all loaded up
    cursor.close()
    db_conn.close()
    print('Count of properties inserted: ' + str(propertycount))
    print('Count of persons inserted: ' + str(personcount))

# db debugging--don't forget conn.commit!
    # newid = insertmap['id']
    #     selectsql = """
    #         SELECT * from property;
    #     """
    #     cursor.execute(selectsql)
    #     print(cursor.fetchone())


if __name__ == '__main__':
    main()


In [36]:
import psycopg2
db_conn = None
def get_db_conn():
    global db_conn
    if db_conn is not None:
        return db_conn
    db_conn = psycopg2.connect(
        dbname="cogdb",
        user="matts207",
        password="12345",
        host="localhost"
    )
    return db_conn


def check_current_owner(parcelid):
    namemap = {}
    get_propid = "SELECT propertyid FROM public.property WHERE parid=%(parcelid)s;"
    inserts = {}
    inserts['parcelid'] = parcelid
    db_conn = get_db_conn()
    cursor = db_conn.cursor()
    cursor.execute(get_propid, inserts)
    propidlist = cursor.fetchall()
    propid = propidlist[0][0]
    get_personid = "SELECT person_personid FROM public.propertyperson WHERE property_propertyid=%(property_propertyid)s;"
    inserts['property_propertyid'] = propid
    cursor.execute(get_personid, inserts)
    personidlist = cursor.fetchall()
    personid = personidlist[0][0]
    get_names = "SELECT fname, lname FROM public.person WHERE personid=%(personid)s"
    inserts['personid'] = personid
    cursor.execute(get_names, inserts)
    names = cursor.fetchall()
    #print(names)
    namemap['fname'] = names[0][0]
    namemap['lname'] = names[0][1]
    return namemap

db = get_db_conn()

props = "SELECT propertyid, parid FROM public.property ORDER BY propertyid;"
propperson = "SELECT person_personid FROM public.propertyperson WHERE property_propertyid = %s"
personSQL = "SELECT lname FROM public.person WHERE "
cursor = db.cursor()
cursor.execute(props)
propidlist = cursor.fetchall()
cursor.close()
for p in propidlist:
    cursor = db.cursor()
    parid = p[1]
    propid = p[0]
    cursor.execute(propperson, (propid, ))
    print(p)
    print(check_current_owner(parid))
    personid = cursor.fetchall()
    print(personid[0][0])

    print('\n')

(170000, '0177P00118000000')
{'fname': None, 'lname': 'TRUDEAU JOHN B WESTCOTT JAY'}
80000


(170001, '0235P00089000000')
{'fname': None, 'lname': 'BONAY ANDREW JR & MARGARETTA O'}
80001


(170002, '0178B00029000000')
{'fname': None, 'lname': 'LINDSAY WILLIAM E'}
80002


(170003, '0178H00296000000')
{'fname': None, 'lname': 'CHIKA MANAGEMENT COMPANY LLC'}
80003


(170004, '0235R00153000000')
{'fname': None, 'lname': 'POLISCHUK LEONID & MARIA (W)'}
80004


(170005, '0177B00137000000')
{'fname': None, 'lname': 'ROGERS JAMES & MARGARET (W)'}
80005


(170006, '0177S00064000000')
{'fname': None, 'lname': 'BARTLETT DAVID MC AFEE'}
80006


(170007, '0178M00032000000')
{'fname': None, 'lname': 'EDMONDS ERIK EDMONDS NIKIA'}
80007


(170008, '0234N00226000000')
{'fname': None, 'lname': 'CRAIG ALBERT M & JOAN B (W)'}
80008


(170009, '0177S00210000000')
{'fname': None, 'lname': 'DAVIS BILLIE'}
80009


(170010, '0234K00054000000')
{'fname': None, 'lname': '2239 HAWTHORNE ASSOCIATES LP'}
80010


(1

{'fname': None, 'lname': 'GOTTSCHALK JENNIFER M'}
80125


(170126, '0235P00270000001')
{'fname': None, 'lname': 'STASINOWSKY WALTER JR & CHERYL A (W)'}
80126


(170127, '0177N00214000000')
{'fname': None, 'lname': 'KOONTZ MARGARET H & MARILYN RITA KOONTZ'}
80127


(170128, '0235J00326000000')
{'fname': None, 'lname': 'KDJ PROPERTIES LLC'}
80128


(170129, '0177K00084000000')
{'fname': None, 'lname': 'COMMUNITY LIVING & SUPPORT SERVICES INC'}
80129


(170130, '0178H00254000000')
{'fname': None, 'lname': 'MORSE TIMOTHY J'}
80130


(170131, '0235A00101000000')
{'fname': None, 'lname': 'SNEAD MELANIE'}
80131


(170132, '0178J00022000000')
{'fname': None, 'lname': 'BONACCI JOHN A & JANICE L (W)'}
80132


(170133, '0177S00212000000')
{'fname': None, 'lname': 'MCELLIGOTT MARGARET & JAMES J CANTWELL'}
80133


(170134, '0178B00254000000')
{'fname': None, 'lname': 'JEFFERSON RICHARD E'}
80134


(170135, '0178L00309000000')
{'fname': None, 'lname': 'CROMBIE THOMAS J'}
80135


(170137, '0177S00258

(170230, '0178D00058000000')
{'fname': None, 'lname': 'GOLDSMITH MARJORIE L'}
80229


(170231, '0178L00009000000')
{'fname': None, 'lname': 'CROMBIE THOMAS J'}
80230


(170232, '0235K00040000000')
{'fname': None, 'lname': 'MONROE JOHN F JR'}
80231


(170233, '0235R00294000000')
{'fname': None, 'lname': 'KLINGENSMITH BRETT B'}
80232


(170234, '0235J00066000000')
{'fname': None, 'lname': 'BURRELLI ANIEL THOMAS'}
80233


(170235, '0235P00087000000')
{'fname': None, 'lname': 'TILLMAN LEROY'}
80234


(170236, '0177M00131000000')
{'fname': None, 'lname': 'DASCHBACH JAMES A'}
80235


(170237, '0178F00007000000')
{'fname': None, 'lname': 'CAPASSO MARY & FRANK (H)'}
80236


(170238, '0177A00105000000')
{'fname': None, 'lname': 'DERDA ROBERT R JR & COLLEEN C (W)'}
80237


(170239, '0178R00130000000')
{'fname': None, 'lname': 'UNION RAILROAD'}
80238


(170240, '0235P00278000000')
{'fname': None, 'lname': 'WOJCICKI CAROL'}
80239


(170241, '0178C00065000000')
{'fname': None, 'lname': 'DEIULIIS FA

(170348, '0178L00172000000')
{'fname': None, 'lname': 'MITCHELL CYNTHIA Y'}
80346


(170349, '0178E00220000000')
{'fname': None, 'lname': 'MCVEAGH HUGH D JR'}
80347


(170350, '0177E00290000000')
{'fname': None, 'lname': 'PENN FAMILY TRUST'}
80348


(170351, '0178M00149000000')
{'fname': None, 'lname': 'ODOMES CATHY R'}
80349


(170352, '0178B00128000000')
{'fname': None, 'lname': 'DASCENZO JOHN F & ROSE'}
80350


(170353, '0177P00022000000')
{'fname': None, 'lname': 'RANDALL BRADLEY T KARLSSON JENNY M (W)'}
80351


(170354, '0178H00040000000')
{'fname': None, 'lname': 'FRANK & ROZ AUTOMOTIVE LLC'}
80352


(170355, '0178L00365000000')
{'fname': None, 'lname': 'PERRONE CHARLES'}
80353


(170356, '0178L00285000000')
{'fname': None, 'lname': 'PEGUES MARLBORO & OLLIE'}
80354


(170357, '0234K00035000000')
{'fname': None, 'lname': 'MAHLER STEPHEN D'}
80355


(170358, '0235R00280000000')
{'fname': None, 'lname': 'BARCZY TIMOTHY J & SUSAN (W)'}
80356


(170359, '0235J00287000000')
{'fname': N

(170475, '0177S00083000000')
{'fname': None, 'lname': 'KING MIRIAM'}
80473


(170476, '0178H00279000000')
{'fname': None, 'lname': 'BOROUGH OF SWISSVALE'}
80474


(170477, '0235F00080000000')
{'fname': None, 'lname': 'YESKO PAUL M & DARLA F (W)'}
80475


(170478, '0177E00274000000')
{'fname': None, 'lname': 'SALIPANTE JEFFREY & LINDA A KING (W)'}
80476


(170479, '0178L00094000000')
{'fname': None, 'lname': 'BEASLEY ALEX W & KESHONA T (W)'}
80477


(170480, '0235J00279000000')
{'fname': None, 'lname': 'OFCANSKY MARJORIE'}
80478


(170481, '0235B00045000000')
{'fname': None, 'lname': 'MADAUS BARBARA M & FRANCIS G (H)'}
80479


(170482, '0234P00064000000')
{'fname': None, 'lname': 'COLE LYNN M'}
80480


(170483, '0235E00370000000')
{'fname': None, 'lname': 'OKOYE CLEMENT M'}
80481


(170484, '0177R00124000000')
{'fname': None, 'lname': 'ARKIN STEVE M'}
80482


(170485, '0177P00276000000')
{'fname': None, 'lname': 'VANDEVENNE ERIK'}
80483


(170486, '0178A00250000000')
{'fname': None, 'ln

...

(173533, '0236C00026000000')
{'fname': None, 'lname': 'PERLA ANTHONY M'}
83527


(173534, '0177B00164000000')
{'fname': None, 'lname': 'WALTON STEVEN D & ROBERTA (W)'}
83528


(173535, '0178A00274000000')
{'fname': None, 'lname': 'ZHANG PENG ZHAO YAN'}
83529


(173536, '0178F00121000000')
{'fname': None, 'lname': 'AGAM HOLDINGS LLC'}
83530


(173537, '0177S00225000000')
{'fname': None, 'lname': 'PERLOV VICTOR & GALINA (W)'}
83531


(173538, '0177R00218000000')
{'fname': None, 'lname': 'COSGROVE MARTIN N & VIRGINIA M'}
83532


(173539, '0178L00181000000')
{'fname': None, 'lname': 'DORSEY ROMANZA M'}
83533


(173540, '0235R00184000000')
{'fname': None, 'lname': 'SMELKO EDWARD J & LEONA C'}
83534


(173541, '0178L00130000000')
{'fname': None, 'lname': 'PALANGIO VERA D (TRUSTEE) (UNDER WILL OF FRANK M PALANGIO) MAKSIMCHUK THERESA ANN (TRUSTEE)(UNDER WILL OF FRANK M PALANGIO)PALANGIO ANTHONY J (TRUSTEE)(UNDER WILL OF FRANK M PALANGIO)'}
83535


(173542, '0177E00211000000')
{'fname': None, '

(173659, '0236C00040000000')
{'fname': None, 'lname': 'EDWARDS LAWRENCE J'}
83653


(173660, '0178H00031000000')
{'fname': None, 'lname': 'HAGENS RENEE L'}
83654


(173661, '0177F00265000000')
{'fname': None, 'lname': 'IVY SCOTT D & SONYA U (W)'}
83655


(173662, '0178L00157000000')
{'fname': None, 'lname': 'SCOTT DENISE M'}
83656


(173663, '0177F00256000000')
{'fname': None, 'lname': 'CHORY MARGARET & EAMON (H)'}
83657


(173664, '0177N00212000000')
{'fname': None, 'lname': 'LYBARGER ROBERT J'}
83658


(173665, '0178M00058000000')
{'fname': None, 'lname': 'LAULETTA JAMES A'}
83659


(173666, '0234N00227000000')
{'fname': None, 'lname': 'CMARADA TED R'}
83660


(173667, '0177B00193000000')
{'fname': None, 'lname': 'DRISCOLL MEGAN L & LOREN A (H)'}
83661


(173668, '0234J00066000000')
{'fname': None, 'lname': 'FINK SARA E'}
83662


(173669, '0177S00211000000')
{'fname': None, 'lname': 'DAVIS BILLIE'}
83663


(173670, '0177S00045000000')
{'fname': None, 'lname': 'BRENNEMAN JACOB & ANNAL

(173783, '0178M00073000000')
{'fname': None, 'lname': 'KLIMCHAK JOHN KLIMCHAK ERIN'}
83777


(173784, '0235R00145000000')
{'fname': None, 'lname': 'KATILIUS JOHN F'}
83778


(173785, '0235E00297000000')
{'fname': None, 'lname': 'STEEL TOWN HOLDINGS'}
83779


(173786, '0235A00208000000')
{'fname': None, 'lname': 'ENGLE MATTHEW TASSONE JAMES'}
83780


(173787, '0178L00146000000')
{'fname': None, 'lname': 'TOWE DENNIS'}
83781


(173788, '0177R00001000000')
{'fname': None, 'lname': 'WILLIAMS SYDNEY C'}
83782


(173789, '0235R00226000000')
{'fname': None, 'lname': 'TASSONE JAMES K ENGLE MATTHEW'}
83783


(173790, '0235J00096000000')
{'fname': None, 'lname': 'MCKINLEY JOSHUA K'}
83784


(173791, '0235L00272000000')
{'fname': None, 'lname': 'MARTY STEPHANIE L'}
83785


(173792, '0234N00140000000')
{'fname': None, 'lname': 'CURRY BERTRAM J'}
83786


(173793, '0235J00205000000')
{'fname': None, 'lname': 'BROWN EDWARD SOURS EMILY MICHELLE'}
83787


(173794, '0177P00296000000')
{'fname': None, 'l

{'fname': None, 'lname': 'WEDNER JAY'}
83901


(173908, '0235K00077000000')
{'fname': None, 'lname': 'NASH ROBERT M'}
83902


(173909, '0236C00001000000')
{'fname': None, 'lname': 'VALKO MARY'}
83903


(173910, '0177R00207000000')
{'fname': None, 'lname': 'CRAWFORD TERESA E & BRUCE (H)'}
83904


(173911, '0178L00238000000')
{'fname': None, 'lname': 'JEFFRIES PAULA MARIE'}
83905


(173912, '0235J00133000000')
{'fname': None, 'lname': 'JACKSON LAVASIA D'}
83906


(173913, '0177P00227000000')
{'fname': None, 'lname': 'DVORCHAK PAUL PENNER JAYME (W)'}
83907


(173914, '0235A00341000000')
{'fname': None, 'lname': 'RAMOS MARTA IRIS'}
83908


(173915, '0234N00167000000')
{'fname': None, 'lname': 'AT ST II LLC SERIES E'}
83909


(173916, '0176P00305000000')
{'fname': None, 'lname': 'LEE PATRICE HOLTZ'}
83910


(173917, '0234R00045000000')
{'fname': None, 'lname': 'SWISSVALE ASSOCIATES'}
83911


(173918, '0234K00013000000')
{'fname': None, 'lname': 'CARTER PEARLMAN & ELLA MAE'}
83912


(173919,

(174038, '0177P00058000000')
{'fname': None, 'lname': 'KETTERER EMILY'}
84032


(174039, '0234P00118000000')
{'fname': None, 'lname': 'PACINDA MICHAEL P CLANCY BRENNA R'}
84033


(174040, '0178A00228000000')
{'fname': None, 'lname': 'KDJ PROPERTIES LLC'}
84034


(174041, '0178G00097000000')
{'fname': None, 'lname': 'PALANGIO CAROL JEAN'}
84035


(174042, '0178A00284000000')
{'fname': None, 'lname': 'OBEDIENTOV ILYA'}
84036


(174043, '0178D00189000000')
{'fname': None, 'lname': 'CURRIE CONSUELO R'}
84037


(174044, '0178M00083000000')
{'fname': None, 'lname': 'CARGO HEATHER R'}
84038


(174045, '0235B00055000000')
{'fname': None, 'lname': 'YOXALL STEVEN R'}
84039


(174046, '0178F00017000000')
{'fname': None, 'lname': 'BUSCH MARTIN F & JEAN L (W)'}
84040


(174047, '0234P00315000000')
{'fname': None, 'lname': 'SWISSVALE ASSOCIATES'}
84041


(174048, '0178L00001000000')
{'fname': None, 'lname': 'LEFEVRE ANDREW'}
84042


(174049, '0178D00100000000')
{'fname': None, 'lname': 'POOLE RACHEL